In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

%matplotlib inline

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.wcs import WCS
from astropy.utils.data import download_file
from astropy.visualization import MinMaxInterval, AsinhStretch, SqrtStretch, ImageNormalize
import astropy.units as u

In [ ]:
import requests
import io

## Load and plot an image from the Astrolabe image server

#### First, let's see what images are available from the image server:

In [ ]:
print(requests.get('http://cuts:8000/cuts/img/list').text)

#### Now, let's select a small image to display. (The following assumes that the `HorseHead.fits` file was listed above. If not, subsitute another file path below).

In [ ]:
# NOTE: This assumes that the HorseHead.fits files exists in the user's `images` directory (as used by the image server)
imgURL = 'http://cuts:8000/cuts/img/fetch?path=/vos/images/HorseHead.fits'

In [ ]:
image_file = download_file(imgURL)
image_hdus = fits.open(image_file)
image_hdus.info()

In [ ]:
image = image_hdus[0].data

In [ ]:
image.shape

In [ ]:
plt.imshow(image, cmap='magma')
plt.colorbar()

## Load and plot a cutout from the Astrolabe image server

In [ ]:
coURL = 'http://cuts:8000/cuts/img/co/cutout_by_filter'
params={'ra': '53.16', 'dec': '-27.78', 'sizeArcSec': '10', 'filter': 'F356W'}

In [ ]:
co_bytes = requests.get(coURL, params)
co_hdus = fits.open(io.BytesIO(co_bytes.content))
co_hdus.info()

In [ ]:
co_hdu = co_hdus[0]
wcs = WCS(co_hdu.header)
cutout = co_hdu.data

In [ ]:
# norm_cutout = ImageNormalize(cutout, interval=MinMaxInterval(), stretch=AsinhStretch())
norm_cutout = ImageNormalize(cutout, interval=MinMaxInterval(), stretch=SqrtStretch())

In [ ]:
fig = plt.figure(figsize=(12,12))
fig.add_subplot(111, projection=wcs)
plt.imshow(cutout, origin='lower', cmap=plt.cm.gray, norm=norm_cutout)
plt.colorbar()
plt.xlabel('RA')
plt.ylabel('Dec')